Input layer is determined by input data, the identity matrix


In [27]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn

import torch.nn.functional as F

torch.set_grad_enabled(False)

torch.set_printoptions(linewidth=120)

In [28]:
class Network(nn.Module):
    def __init__(self):
        super(Network,self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1,out_channels=6,kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6,out_channels=12,kernel_size=5)

        self.fc1 = nn.Linear(in_features=12*4*4, out_features=120) 
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)#Fully connected/linear layer


    def forward(self,t):
        #1 input layer
        t=t

        #2 hidden convolutional layer
        t = self.conv1(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2,stride = 2)

        # 3 hidden conv layer 2, comprised of collection of weights and operations
        # Weights are stored in neural network modular layer class instance
        t = self.conv2(t)
        t = F.relu(t) #pure operation, no Weights, activation
        t = F.max_pool2d(t, kernel_size=2,stride = 2) #pure operation, no Weights

        # 4 Linear Layer
        t = t.reshape(-1,12*4*4) #4 by 4 is the height and width 
        t = self.fc1(t)
        t = F.relu(t)

        # 5 Linear layer
        t = self.fc2(t)
        t = F.relu(t)

        # 6 Linear Layer
        t = self.out(t)
        #t = F.softmax(t,dim=1)

        return t

In [29]:
train_set = torchvision.datasets.FashionMNIST(
    root ='./data/FashionMNIST'
    ,train=True
    ,download=True
    ,transform=transforms.Compose([transforms.ToTensor()
    ])
)

In [30]:
network = Network()

In [33]:
#Single sample prediction

sample = next(iter(train_set))
image, label = sample
image.unsqueeze(0).shape
pred = network(image.unsqueeze(0))
print(pred.shape) #1 image and10 prediction classes each has a value
#print(pred)
print(F.softmax(pred,dim=1)) #most predictions came in at around 10%
print(pred.argmax(dim=1))
print(label) 
print("Model predicts coat (4) whilst the image is an ankle boot (9)")

torch.Size([1, 10])
tensor([[0.1080, 0.1106, 0.0945, 0.0882, 0.1132, 0.1108, 0.1084, 0.0868, 0.0927, 0.0868]])
tensor([4])
9
Model predicts coat (4) whilst the image is an ankle boot (9)


In [37]:
#Batch sample prediction

data_loader = torch.utils.data.DataLoader(train_set, batch_size=10)
batch = next(iter(data_loader))
images, labels = batch
preds = network(images)
print(preds.argmax(dim=1))
print(labels)
print(preds.argmax(dim=1).eq(labels))


tensor([4, 4, 4, 4, 4, 1, 4, 4, 4, 4])
tensor([9, 0, 0, 3, 0, 2, 7, 2, 5, 5])
tensor([False, False, False, False, False, False, False, False, False, False])
